In [32]:
from pymongo import MongoClient
import pandas as pd
import pymongo
import folium

In [33]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies
data = db.data_companies_clean.find()

In [34]:
data_comp = pd.DataFrame(data)
display(data_comp.head())

,_id,alias_list,category_code,description,founded_year,lat,lng,main_office,total_money_raised
0,5d2633ac797d7972f3178d9e,,web,Life long learning content,2007,34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,...",$21M
1,5d2633ac797d7972f3178d9f,None,web,None,2007,37.269175,-119.306607,"{'type': 'Point', 'coordinates': [-119.306607,...",$100k
2,5d2633ac797d7972f3178da0,,web,Social network for desktop software,2007,52.374523,4.894862,"{'type': 'Point', 'coordinates': [4.8948623, 5...",$1M
3,5d2633ac797d7972f3178da1,,web,Video guide for hotels,2007,48.856667,2.350987,"{'type': 'Point', 'coordinates': [2.350987, 48...",$13.8M
4,5d2633ac797d7972f3178da2,None,web,None,2007,40.010492,-105.276843,"{'type': 'Point', 'coordinates': [-105.276843,...",$315k


In [35]:
# def findNear(df,radio_max_meters = 2000):
#     location = db.data_companies_clean.find({'main_office':{
#         '$near':{
#             '$geometry':df,
#             '$maxDistance': radio_max_meters,
#         }
#     }
#     })
#     df1 = pd.DataFrame(location)
#     return len(df1)
# near_offices = findNear(data_comp['main_office'][0])

In [36]:
def findNear(df, radio_max_meters = 20):
    lst =[]
    for i in range(len(df)):
        location = db.data_companies_clean.find({'main_office':{
                                                 '$near': {
                                                     '$geometry': df[i],
                                                     '$maxDistance': radio_max_meters,
                                                 }
                                                }
                                                })
        
        df2 = pd.DataFrame(location)
        lst.append(len(df2))    
    return lst

near_offices = findNear(data_comp['main_office'])
len(near_offices)

154

In [37]:
data_comp['Near offices'] = pd.DataFrame(near_offices)

In [38]:
data_comp.head()

,_id,alias_list,category_code,description,founded_year,lat,lng,main_office,total_money_raised,Near offices
0,5d2633ac797d7972f3178d9e,,web,Life long learning content,2007,34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,...",$21M,1
1,5d2633ac797d7972f3178d9f,None,web,None,2007,37.269175,-119.306607,"{'type': 'Point', 'coordinates': [-119.306607,...",$100k,2
2,5d2633ac797d7972f3178da0,,web,Social network for desktop software,2007,52.374523,4.894862,"{'type': 'Point', 'coordinates': [4.8948623, 5...",$1M,1
3,5d2633ac797d7972f3178da1,,web,Video guide for hotels,2007,48.856667,2.350987,"{'type': 'Point', 'coordinates': [2.350987, 48...",$13.8M,2
4,5d2633ac797d7972f3178da2,None,web,None,2007,40.010492,-105.276843,"{'type': 'Point', 'coordinates': [-105.276843,...",$315k,2


In [39]:
base_map = folium.Map(zoom_start=15)

In [40]:
a = list(zip(data_comp['lat'], data_comp['lng']))
for i in a:
    b = folium.Marker(location=[i[0],i[1]]).add_to(base_map) 
    
base_map

In [46]:
from folium.plugins import FastMarkerCluster
base_map_two = folium.Map(zoom_start=15)





In [53]:
FastMarkerCluster(data=list(zip(data_comp['lat'].values, data_comp['lng'].values))).add_to(base_map_two)
base_map_two

In [90]:
import json
import requests
from dotenv import load_dotenv
import os

In [141]:
load_dotenv()
if not "KEYG" in os.environ:
    raise ValueError("You should pass a Google Key")
    
google_key = os.environ["KEYG"]

In [142]:
BASE_URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch'
res = requests.get("{}/json?location=34.017606,-118.487267&radius=1500&type=school&key={}".format(BASE_URL, google_key))





In [150]:
locat = []
for i in a:
    locat.append(i)



'(34.017606, -118.487267)'

In [152]:
asd = []
for i in locat:
    asd.append(requests.get("{}/json?location={},{}&radius=1500&type=school&key={}".format(BASE_URL, i[0], i[1], google_key)))
                 


In [155]:
asd_jsons = []
for i in asd:
    asd_jsons.append(i.json())
asd_jsons

[{'html_attributions': [],
  'next_page_token': 'CqQCHQEAAMGvxAmAeqITT0EqIoI3sI1nvPBgdwE8X4852iMwP2H6A6-qRoiGzPEIja_pcDipcJTIwM8xplGA3LDF5mhazLBn_4WScu88z89FWPCcqpwAdAc_5nlFqt2B8padGZ6rvUi__Wr0r1bOfhXS7TUp6w59ZbeQ62X41U5h_sLLQrGpmTMxqpbtFPa7YXgcMzm6r97tHEx3u0IWkfvX-YSwO7e0RhtW3W-tlkKIej-K5V2T0SFU81hWDZFq1LG4fYH5qpxFXxcXt_MAV9K0H2SA4wYY42uoxdAr_JzsudK4E01JH2Pch2P7Ez30xJve3RnzzbbYdd_YZc125GySuxszvx10qB4xLkx-oj1_TW2h8AEL1eqGI0sIPYNirSv17dmOJxIQL-xjpZnthsFevdt9szrAZRoUJZ3TCPEZ4gOlPsDAD5Uc0eqXgIs',
  'results': [{'geometry': {'location': {'lat': 34.0120677,
      'lng': -118.4868928},
     'viewport': {'northeast': {'lat': 34.01532449999999,
       'lng': -118.48467015},
      'southwest': {'lat': 34.01098210000001, 'lng': -118.48882395}}},
    'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/school-71.png',
    'id': 'ea40584f8e4d492a50617ccdc7d0e0a57a18a1f8',
    'name': 'Santa Monica High School',
    'photos': [{'height': 1932,
      'html_attributions': ['<a href="https://map

In [ ]:
#https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY

In [75]:
BASE_URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='


In [71]:
res = requests.get('{}-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key={}'.format(BASE_URL, google_key)


SyntaxError: unexpected EOF while parsing (<ipython-input-71-ebc51e01c322>, line 2)

In [ ]:
BASE_URL = “https://maps.googleapis.com/maps/api/place/nearbysearch”